In [11]:
from random import random, seed
import numpy as np
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt

In [ ]:
# Importing various packages
from random import random, seed
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor

n = 100
x = 2*np.random.rand(n,1)
y = 4+3*x+np.random.randn(n,1)

X = np.c_[np.ones((n,1)), x]
beta_linreg = np.linalg.inv(X.T @ X) @ (X.T @ y)
print(beta_linreg)
sgdreg = SGDRegressor(max_iter = 50, penalty=None, eta0=0.1)
sgdreg.fit(x,y.ravel())
print(sgdreg.intercept_, sgdreg.coef_)

In [12]:
# Loading dataset and creating Design Matrix
np.random.seed(0)        #create same seed for random number every time
cancer=load_breast_cancer()      #Download breast cancer dataset
inputs=cancer.data                     #Feature matrix of 569 rows (samples) and 30 columns (parameters)
outputs=cancer.target                  #Label array of 569 rows (0 for benign and 1 for malignant)
labels=cancer.feature_names[0:30]

#Print information about the datasets
print('The content of the breast cancer dataset is:')
print(labels)
print('-------------------------')
print("inputs =  " + str(inputs.shape))
print("outputs =  " + str(outputs.shape))
print("labels =  "+ str(labels.shape))

def standard_scale(data):
    mean = np.mean(data, axis=0)
    std_dev = np.std(data, axis=0)
    scaled_data = (data - mean) / std_dev
    return scaled_data, mean, std_dev

x = inputs 
x, mean, std_dev = standard_scale(x)
y = outputs
temp1 = np.reshape(x[:,1], (len(x[:,1]),1))
temp2 = np.reshape(x[:,2], (len(x[:,2]),1))
X = np.hstack((temp1, temp2))      
temp = np.reshape(x[:,5], (len(x[:,5]),1))
X = np.hstack((X,temp))       
temp = np.reshape(x[:,8], (len(x[:,8]),1))
X = np.hstack((X,temp))
y = y.reshape(y.shape[0], 1)
#X_train, X_test, y_train, y_test = train_test_split_numpy(X, y, 0.7, 0.3)   #Split datasets into training and testing
del temp1,temp2,temp
#y_train_onehot = to_categorical_numpy(y_train)
#y_test_onehot = to_categorical_numpy(y_test)

The content of the breast cancer dataset is:
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
-------------------------
inputs =  (569, 30)
outputs =  (569,)
labels =  (30,)


In [9]:
#Defining Cost function
def CostCrossEntropy(target):
    
    def func(X):
        return -(1.0 / target.size) * np.sum(target * np.log(X + 10e-10))

    return func


In [4]:
#Returns mean squared error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

#Returns accuracy score, although this one is mostly 0
def accuracy_score_numpy(Y_test, Y_pred):
    return np.sum(Y_test == Y_pred) / len(Y_test)


In [13]:
# Using Autograd to calculate gradients using ADAM and Stochastic Gradient descent

#X = np.c_[np.ones((n,1)), x, x*x]
XT_X = X.T @ X
theta_linreg = np.linalg.pinv(XT_X) @ (X.T @ y)
print("Own inversion")
print(theta_linreg)


# Note that we request the derivative wrt third argument (theta, 2 here)
training_gradient = grad(CostCrossEnthropy)
# Define parameters for Stochastic Gradient Descent
n_epochs = [1,10,100,1000]
M = 10   #size of each minibatch
m = int(n/M) #number of minibatches
# Guess for unknown parameters theta
theta = np.random.randn(3,1)
lmbd = 0.01

# Value for learning rate
eta_vals = [1e-4, 1e-3, 1e-2, 1e-1]
# Value for parameters beta1 and beta2, see https://arxiv.org/abs/1412.6980
beta1 = 0.9
beta2 = 0.999
# Including AdaGrad parameter to avoid possible division by zero
delta  = 1e-7
iter = 0
for epoch in range(n_epochs):
    first_moment = 0.0
    second_moment = 0.0
    iter += 1
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X[random_index:random_index+M]
        yi = y[random_index:random_index+M]
        gradients = (1.0/M)*training_gradient(yi, xi, theta)
        # Computing moments first
        first_moment = beta1*first_moment + (1-beta1)*gradients
        second_moment = beta2*second_moment+(1-beta2)*gradients*gradients
        first_term = first_moment/(1.0-beta1**iter)
        second_term = second_moment/(1.0-beta2**iter)
	# Scaling with rho the new and the previous results
        update = eta*first_term/(np.sqrt(second_term)+delta)
        theta -= update
print("theta from own ADAM")
print(theta)

Own inversion
[[-0.08685965]
 [-0.27459236]
 [-0.08268545]
 [-0.0535042 ]]


NameError: name 'CostCrossEnthropy' is not defined

In [ ]:
fontsize = 12   

train_accuracy *= 100
test_accuracy *= 100

fig, ax = plt.subplots(figsize = (6, 6))
sns.heatmap(train_accuracy, annot=True, ax=ax, cmap="viridis", xticklabels=epochs, yticklabels=eta_vals, fmt=".1f",  annot_kws={"size": fontsize},  vmin=0, vmax=100)
ax.set_title("Training Accuracy Own Code [%]", fontsize=fontsize)
ax.set_ylabel("Learning Rate", fontsize=fontsize)
ax.set_xlabel("Epochs", fontsize=fontsize)
plt.tick_params(axis='both', labelsize=fontsize)
plt.savefig('/home/Kristen/Documents/FYS4155/Excecises/C_own_EpoLR_Train.pdf', format='pdf', bbox_inches="tight")
plt.show()

fig, ax = plt.subplots(figsize = (6, 6))
sns.heatmap(test_accuracy, annot=True, ax=ax, cmap="viridis", xticklabels=epochs, yticklabels=eta_vals, fmt=".1f",  annot_kws={"size": fontsize},  vmin=0, vmax=100)
ax.set_title("Test Accuracy Own Code [%]", fontsize=fontsize)
ax.set_ylabel("Learning Rate", fontsize=fontsize)
ax.set_xlabel("Epochs", fontsize=fontsize)
plt.tick_params(axis='both', labelsize=fontsize)
plt.savefig('/home/Kristen/Documents/FYS4155/Excecises/C_own_EpoLR_Test.pdf', format='pdf', bbox_inches="tight")
plt.show()

In [ ]:
#Stochastic gradient descent
n_epochs = int(n/5)
M = int(100)
eta = 0.1
#SGD function takes x,y, number of epochs, size of minibatch, number datapoints, and the name of the learning rate modifier
beta, beta_linreg = SGD(x,y, n_epochs, M, n, "ADAGRA", eta)
xbnew = np.c_[np.ones((len(x),1)), x, x**2, x**3, x**4, x**5, x**6]
ypred = xbnew @ beta
ypred2 = xbnew @ beta_linreg
plt.plot(x,ypred, 'ro')
plt.plot(x,y)

In [5]:
#main

#initialize parameters. Might need some normalization.

poly_deg = 3 #polynomial degree
xlim = 1.0 #range of x values goes from 0 to xlim
np.random.seed(42) #creates random seed to have reproducible results

n = 1000 # number of data points
step = 1/n # step size in x
x = np.arange(0, xlim, step) # create data points
target = FrankeFunction(x) # create y-data
target = target.reshape(target.shape[0], 1) # reshaping y-data
y = target # creating a copy of the target

from sklearn.model_selection import train_test_split


X = createX(x,poly_deg) # creating design matrix
X_train, X_test, y_train, y_test = train_test_split(X,y) # splitting data


n_output_nodes = 1 #s et to 1 for regression
n_input_nodes = X_train.shape[1] # set number of input nodes
n_hidden_nodes = 5 # set number of hidden nodes

FFNN1 = FFNN(dimensions = [n_input_nodes,n_hidden_nodes,n_output_nodes],hidden_func = sigmoid) #creates an object of the neural network


scheduler = Adagrad(eta=1e-3) # creates a schedueler which controls the learning rate decay. Set this equal to the scheduele mode, i.e. Adagrad, Constant, ADAM, with the paramaters requiered for this scheduele

varlist = np.logspace(-9,-.0005, 1) #list of variables to be tested. Set third float to 1 if nothing is to be tested
MSElist = [] #list of MSE. Not sure if this is usefull.

M = 10
lmbd_vals = [1E2,2E2,3E2,4E2,5E3] 
eta_vals = [1e-20, 1e-16, 1e-14, 1e-10, 1e-1] #Can be adjusted, but code gets very slow for eta below 1e-10
epochs = 100

DNN_numpy = np.zeros((len(eta_vals), len(lmbd_vals)), dtype=object)
train_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))
test_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))

for i, etaa in enumerate(eta_vals):
    for j, lmbd in enumerate(lmbd_vals): # starts a loop to test variables.
        FFNN1.fit(X_train,y_train, Adagrad(eta = (etaa)), epochs = int(lmbd)) #train data to the fit. Using the training data here, not sure if this is the best option. Many parameters can be set to optimize the fit, such as eta, lambda, momentum.
        #DNN_numpy[i][j] = fitted
        train_pred = FFNN1.predict(X_train) #creates a prediction from the training. Hopefully this is working, but not completely sure.
        test_pred = FFNN1.predict(X_test)
        #score = FFNN1._accuracy(y,y2) #supposed to give accuracy score of the fit, returns 0

        MSE_train = mean_squared_error(y_train, train_pred) #return MSE of the fit.
        MSE_test = mean_squared_error(y_test, test_pred)
        
        train_accuracy[i][j] = MSE_train
        test_accuracy[i][j] = MSE_test
        print("Learning rate  = ", etaa)
        print("Lambda = ", lmbd)
        print("Accuracy score on test set: ", mean_squared_error(y_test, test_pred))


# Grid search and visualization own code
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set()

M = 10
lmbd_vals = [1e-20, 1e-15, 1e-10, 1e-5, 1e0] 
eta_vals = [1e-20, 1e-16, 1e-14, 1e-10, 1e-6] #Can be adjusted, but code gets very slow for eta below 1e-10
epochs = 1000000

DNN_numpy = np.zeros((len(eta_vals), len(lmbd_vals)), dtype=object)

# grid search
for i, eta in enumerate(eta_vals):
    for j, lmbd in enumerate(lmbd_vals):
        FFNN1.reset_weights() #important to reset weights, as to not generate data based on previous runs
        scores = FFNN1.fit(X_train,y_train, Adagrad(eta = eta), epochs = 1000) #train data to the fit. Using the training data here, not sure if this is the best option. Many parameters can be set to optimize the fit, such as eta, lambda, momentum.   
        DNN_numpy[i][j] = FFNN1
        test_predict = FFNN1.predict(X) #creates a prediction from the training.
        score = FFNN1._accuracy(y,test_predict)
        print("Learning rate  = ", eta)
        print("Lambda = ", lmbd)
        print("Accuracy score on test set: ", mean_squared_error(y, test_predict))

train_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))
test_accuracy = np.zeros((len(eta_vals), len(lmbd_vals)))

for i in range(len(eta_vals)):
    for j in range(len(lmbd_vals)):
        dnn = DNN_numpy[i][j]
        train_pred = dnn.predict(X_train) 
        test_pred = dnn.predict(X_test)
        train_accuracy[i][j] = mean_squared_error(y_train, train_pred)
        test_accuracy[i][j] = mean_squared_error(y_test, test_pred)

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(train_accuracy, annot=True, ax=ax, cmap="viridis_r", xticklabels=lmbd_vals, yticklabels=eta_vals)
ax.set_title("Training Accuracy")
ax.set_ylabel("eta")
ax.set_xlabel("lambda")
plt.show()

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(test_accuracy, annot=True, ax=ax, cmap="viridis_r", xticklabels=lmbd_vals, yticklabels=eta_vals)
ax.set_title("Test Accuracy")
ax.set_ylabel("eta")
ax.set_xlabel("lambda")
plt.show()


Adagrad: Eta=1e-20, Lambda=0
  [=======================================>] 100.0% | train_error: 2.46 Learning rate  =  1e-20
Lambda =  100.0
Accuracy score on test set:  0.5356897750042423
Adagrad: Eta=1e-20, Lambda=0
  [=======================================>] 100.0% | train_error: 2.62  Learning rate  =  1e-20
Lambda =  200.0
Accuracy score on test set:  0.5356897750042423
Adagrad: Eta=1e-20, Lambda=0
  [=======================================>] 100.0% | train_error: 2.42  Learning rate  =  1e-20
Lambda =  300.0
Accuracy score on test set:  0.5356897750042423
Adagrad: Eta=1e-20, Lambda=0
  [=======================================>] 100.0% | train_error: 2.51  Learning rate  =  1e-20
Lambda =  400.0
Accuracy score on test set:  0.5356897750042423
Adagrad: Eta=1e-20, Lambda=0
  [=======================================>] 100.0% | train_error: 2.52   Learning rate  =  1e-20
Lambda =  5000.0
Accuracy score on test set:  0.5356897750042423
Adagrad: Eta=1e-16, Lambda=0
  [=================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



  [=====================================>--] 96.60% | train_error: 2.43 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
def sigmoid(X):
    try:
        return 1.0 / (1 + np.exp(-X))
    except FloatingPointError:
        return np.where(X > np.zeros(X.shape), np.ones(X.shape), np.zeros(X.shape))


def softmax(X):
    X = X - np.max(X, axis=-1, keepdims=True)
    delta = 10e-10
    return np.exp(X) / (np.sum(np.exp(X), axis=-1, keepdims=True) + delta)


def RELU(X):
    return np.where(X > np.zeros(X.shape), X, np.zeros(X.shape))


def LRELU(X):
    delta = 10e-4
    return np.where(X > np.zeros(X.shape), X, delta * X)


def SGD(x,y,n_epochs,M,n, learningMod, eta):
    X = np.c_[np.ones((n,1)),x,x**2,x**3, x**4, x**5, x**6]
    m = int(n/M)
    t0 = M
    t1 = n_epochs
    beta_linreg = np.linalg.inv(X.T @ X) @ X.T @ y
    beta = np.random.randn(len(X[1]),1)
    training_gradient = grad(cost)
    change = 0.0
    delta = 1E-8
    momentum = 0.2
    beta1 = 0.9
    beta2 = 0.999
    rho = 0.99
    iter = 0
    
    for epoch in range(n_epochs):
        Giter = 0.0
        first_moment = 0.0
        second_moment = 0.0
        iter += 1
        for i in range(m):
            random_index = M*np.random.randint(m)
            xi = X[random_index:random_index+M]
            yi = y[random_index:random_index+M]
            
            gradients = gradient(xi,yi,M,beta)
            #gradients = training_gradient(xi,yi,beta)
            
            if(learningMod == "ADAGRAD"):
                Giter += gradients**2
                new_change = (eta*gradients.reshape(beta.shape))/(delta+np.sqrt(Giter)) + momentum * change
            if(learningMod == "RMSprop"):
                Giter = (rho*Giter+(1-rho)*gradients*gradients)
                new_change = (eta*gradients.reshape(beta.shape))/(delta+np.sqrt(Giter)) + momentum * change
            if(learningMod == "ADAM"):
                first_moment = beta1*first_moment + (1-beta1)*gradients
                second_moment = beta2*second_moment+(1-beta2)*gradients*gradients
                first_term = first_moment/(1.0-beta1**iter)
                second_term = second_moment/(1.0-beta2**iter)
                new_change = eta*first_term/(np.sqrt(second_term)+delta) + momentum * change
            else:
                new_change = eta*gradients.reshape(beta.shape) + momentum * change
            beta -= new_change
            change = new_change
            
            
    return(beta, beta_linreg)